## EX1

Uma empresa de comida caninda produz dois tipos de rações: Tobi e Rex. Para a manufatura das rações são utilizados cereais e carnes. Sabe-se que:

- A ração Tobi utiliza 5kg de cereais e 1kg de carne, e a ração Rex utiliza 4kg de carne e 2kg de cereais

- O pacote de ração Tobi custa R$20,00 e o pacote do Rex custa R$30,00

- O kg de carne custa R$4,00 e o de cereais custa R$1,00

- Estão disponíveis por mês 10_000kg de carne e 30_000kg de cereais 

Deseja-se saber qual a quantidade de cada ração a produzir de modo a maximizar o lucro

###  Modelo Matemático

#### Variáveis 



t = quantidade da ração Tobi a ser produzida 

r = quantidade da ração Rex a ser produzida

#### Função Objetivo

Max Z = Valor de cada pacote - custos de produção

$Max \quad Z = 20*t+30*r - (t*5*1+t*1*4) - (r*2*1+r*4*4)$

#### Restrições

- Limite quantidade de cereal

$t*5 + 2*r = 30000$

- Limite quantidade de carne

$1*t + 4*r = 10000$




In [1]:
import pyomo.environ as pyo
import time

In [2]:
# Criação do modelo
model = pyo.ConcreteModel("Racao")

# Tipos de ração:
model.R = pyo.Set(initialize=["Tobi", "Rex"])

# Tipos de material
model.M = pyo.Set(initialize=["Cereal", "Carne"])

# ------------------------------
# Parâmetros
# ------------------------------
# Custos material
material = {"Cereal":1, "Carne":4}
model.c_material = pyo.Param(model.M,initialize=material)

# Qtd material por pacote
qtd_material = {("Tobi","Cereal"):5, ("Rex","Cereal"):2,("Tobi","Carne"):1, ("Rex","Carne"):4}
model.q_material = pyo.Param(model.R,model.M,initialize=qtd_material)

# Material disponível

estoque = {"Cereal":30000, "Carne":10000}
model.e_material = pyo.Param(model.M,initialize=estoque)

# Custos pacote
valor_venda = {"Tobi":20, "Rex":30}
model.v_pacote = pyo.Param(model.R,initialize=valor_venda)

# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.R,domain=pyo.NonNegativeIntegers)

# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    # Valor pacote
    valor_pacote = sum(
        model.x[i] * model.v_pacote[i]
        for i in model.R
    )

    # Custo material
    custo = sum(
        model.x[i] * model.c_material[m] * model.q_material[i,m]
        for i in model.R
        for m in model.M
    )

    return valor_pacote + custo


model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# ------------------------------
# Restrições
# ------------------------------

# 1. Limite Estoque
def estoque(model, m):
    return sum(model.x[i] * model.q_material[i,m] for i in model.R) <= model.e_material[m]


model.CapRef = pyo.Constraint(model.M, rule=estoque)



In [3]:
# Criação do modelo
model = pyo.ConcreteModel("Racao")

In [4]:
# Tipos de ração:
model.R = pyo.Set(initialize=["Tobi", "Rex"])

# Tipos de material
model.M = pyo.Set(initialize=["Cereal", "Carne"])

In [5]:
# ------------------------------
# Parâmetros
# ------------------------------
# Custos material
material = {"Cereal":1, "Carne":4}
model.c_material = pyo.Param(model.M,initialize=material)

# Qtd material por pacote
qtd_material = {("Tobi","Cereal"):5, ("Rex","Cereal"):2,("Tobi","Carne"):1, ("Rex","Carne"):4}
model.q_material = pyo.Param(model.R,model.M,initialize=qtd_material)

# Material disponível

estoque = {"Cereal":30000, "Carne":10000}
model.e_material = pyo.Param(model.M,initialize=estoque)

# Custos pacote
valor_venda = {"Tobi":20, "Rex":30}
model.v_pacote = pyo.Param(model.R,initialize=valor_venda)



In [6]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.R,domain=pyo.NonNegativeIntegers)

In [7]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    # Valor pacote
    valor_pacote = sum(
        model.x[i] * model.v_pacote[i]
        for i in model.R
    )

    # Custo material
    custo = sum(
        model.x[i] * model.c_material[m] * model.q_material[i,m]
        for i in model.R
        for m in model.M
    )

    return valor_pacote + custo


model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

In [8]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite Estoque
def estoque(model, m):
    return sum(model.x[i] * model.q_material[i,m] for i in model.R) <= model.e_material[m]


model.CapRef = pyo.Constraint(model.M, rule=estoque)


In [9]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex1.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 2 rows; 2 cols; 4 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [3e+01, 5e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+04, 3e+04]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   4 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.00% 

In [10]:
for i in model.R:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade da ração {i} : {val:.2f}")

  Quantidade da ração Tobi : 5555.00
  Quantidade da ração Rex : 1111.00
